IMPORTS

In [5]:
import pandas as pd
import numpy as np

In [6]:
from scipy.stats import binom,poisson,geom,hypergeom,uniform,expon, gamma, beta,chi2,t,pareto,lognorm

In [7]:
from statsmodels.tsa.arima_process import arma_generate_sample

In [8]:
from random import randrange, random
import math

In [9]:
from scipy.signal import lfilter

CREACIÓN SERIES TEMPORALES SEGÚN: PERIODOS / FIN

In [10]:
# A partir de una fecha de inicio y fecha de fin, devuelve la serie temporal desde la fecha de inicio a la fecha de fin con frequencias posibles:
# B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds

def series_fin(inicio, fin,freq):
    serie = pd.date_range(start=inicio, end=fin, freq=freq)
    return serie 

In [11]:
# A partir de una fecha de inicio y el número de periodos, devuelve la serie temporal desde la fecha de inicio a la fecha de fin con frequencias posibles:
# B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds

def series_periodos(inicio, periodos, freq): 
    serie = pd.date_range(start=inicio, periods=periodos, freq=freq)
    return serie

MODELOS DE TENDENCIA DETERMINISTA

TRES TIPOS DE TENDENCIA:

In [12]:
def tendencia_lineal (a,b,t):
    return a + b * t 

In [13]:
def tendencia_cuadratica (a,b,c,t):
    return a + b * t + c * t * t

In [14]:
def tendencia_determinista_lineal (a,b,t,e=0):
    return tendencia_lineal(a,b,t) + e

In [15]:
def tendencia_determinista_cuadratica(a,b,c,t,e=0):
    return tendencia_cuadratica(a,b,c,t) + e

In [16]:
def tendencia_determinista_exponencial(a,b,t,e=0):
    return math.exp(a+b*t+e)

Parámetros del modelo:
    - params: indica los parámetros de la tendencia.
    - tipo: Valores posibles:
        1. Tendencia determinista lineal (2 parámetros)
        2. Tendencia determinista cuadrática (3 parámetros)
        3. Tendencia determinista exponencial (2 parámetros)
    - num_datos: número de datos a generar
    - coef_error: indica el coeficiente de error (a mayor coeficiente, menor estacionariedad). Si el coef_error=0, los datos son estacionarios. 

In [17]:
def tendencia_det(params,tipo,num_datos,coef_error=0):
    
    datos = np.zeros(num_datos)
    
    for t in range(0,num_datos):
        e = random()*coef_error
        
        if tipo==1:
            datos[t] = tendencia_determinista_lineal(params[0],params[1],t,e)
        elif tipo==2:
            datos[t] = tendencia_determinista_cuadratica(params[0],params[1],params[2],t,e)
        elif tipo==3:
            datos[t] = tendencia_determinista_exponencial(params[0],params[1],t,e)
            
    return datos   

CREACIÓN DE DATOS SEGÚN CIERTAS DISTRIBUCIONES:

Parámetros:
- num_datos : numéro de datos a generar 
- Distr, params : Indica la distribución a seguir y los parámetros de esta distribución. Casos:
    1. Los parámetros son la media y la desv_típica de los datos. Sigue una distribución normal.
    2. Los parámetros son n (nº pruebas) y p (probabilidad éxito) de una distribución binomial. Además, podemos añadir otro parámetro que indica la localización de los datos.
    3. El parámetro es mu de una distribución poisson. Además, podemos añadir otro parámetro que indica la localización de los datos.
    4. El parámetro es p (probabilidad éxito) de una distribución geométrica. Además, podemos añadir otro parámetro que indica la localización de los datos.
    5. Los parámetros son M (tamaño población), n (nº aciertos) y N (tamño muestra) de una distribución hipergeométrica. Además, podemos añadir otro parámetro que indica la localización de los datos.
    6. El parámetro es el valor constante que toma los datos.
    7. Sigue una distribución uniforme. Puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    8. El parámetro es s de una distribución lognormal. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    9. Sigue una distribución exponencial. Puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    10. El parámetro es a de una distribución gamma. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    11. Los parámetros son a y b de una distribución beta. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    12. El parámetro es df de una distribución chi cuadrado. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    13. El parámetro es t de una distribución t-student. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    14. El parámetro es b de una distribución pareto. Además, podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    15. Sigue una distribución lineal descendente cuyos parámetros son el dato inicial y la pendiente (en valor absoluto). El mínimo valor posible es cero, si se alcanza los datos se mantienen constante con ese valor. 
    16. Sigue una distribución lineal ascendente cuyos parámetros son el dato inicial y la pendiente. 
    17. Obtienes números aleatorios cuyos valores oscilan entre el primer parámetro y el segundo.

In [18]:
def crear_datos(distr,params,num_datos):
    
    if distr == 1 :
        datos = np.random.normal(params[0],params[1],num_datos)
        
    elif distr ==2 :
        if len(params)==2:
            datos = binom.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = binom.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr== 3 :
        if len(params)==1:
            datos = poisson.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = poisson.rvs(params[0],params[1],size=num_datos)
            
    elif distr == 4 :
        if len(params)==1:
            datos = geom.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = geom.rvs(params[0],params[1],size=num_datos)
            
    elif distr == 5:
        if len(params)==3:
            datos = hypergeom.rvs(params[0],params[1],params[2],size=num_datos)
        elif len(params) == 4:
            datos = hypergeom.rvs(params[0],params[1],params[2],params[3],size=num_datos)
            
    elif distr == 6: 
        datos = np.zeros(num_datos) + params[0]
        
    elif distr == 7:
        if len(params)==0:
            datos = uniform.rvs(size=num_datos)
        elif len(params)==1:
            datos = uniform.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = uniform.rvs(params[0],params[1],size=num_datos)
            
    elif distr == 8:
        if len(params)==1:
            datos = lognorm.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = lognorm.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = lognorm.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 9: 
        if len(params)==0:
            datos = expon.rvs(size=num_datos)
        elif len(params)==1:
            datos = expon.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = expon.rvs(params[0],params[1],size=num_datos)
            
    elif distr == 10: 
        if len(params)==1:
            datos = gamma.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = gamma.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = gamma.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 11: 
        if len(params)==2:
            datos = beta.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = beta.rvs(params[0],params[1],params[2],size=num_datos)
        elif len(params) == 4:
            datos = beta.rvs(params[0],params[1],params[2],params[3],size=num_datos)
            
    elif distr == 12: 
        if len(params)==1:
            datos = chi2.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = chi2.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = chi2.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 13: 
        if len(params)==1:
            datos = t.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = t.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = t.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 14: 
        if len(params)==1:
            datos = pareto.rvs(params[0],size=num_datos)
        elif len(params) == 2:
            datos = pareto.rvs(params[0],params[1],size=num_datos)
        elif len(params) == 3:
            datos = pareto.rvs(params[0],params[1],params[2],size=num_datos)
            
    elif distr == 15:
        datos = np.zeros(num_datos)
        datos[0]= params[0]
        i=1
        while datos[i-1]>0 and i<num_datos:
            datos[i] = datos[i-1] - params[1]
            i= i+1
            
    elif distr == 16:
        datos = np.zeros(num_datos)
        datos[0] = params[0]
        for i in range(1,num_datos):
            datos[i] = datos[i-1] + params[1]
    
    elif distr == 17:
        datos= np.zeros(num_datos)
        for i in range(0,num_datos):
            datos[i] = randrange(params[0],params[1])
        
    return datos

MODELOS PERIÓDICOS

CREACIÓN DE MODELOS DE DATOS PERIÓDICOS A PARTIR DE UNA DISTRIBUCIÓN SABIENDO EL NÚMERO DE DATOS A GENERAR Y ALGUNA DE LAS SIGUIENTES OPCIONES:
    1. AMPLITUD PERIODO
    2. NÚMERO DE PERIODOS

Parámetros:
    - distr, params : indica la distribución que van a seguir los datos en cada periodo y los parámetros de esta distribución.
    - num_datos: indica el número de datos a crear.
    - periodo: indica el número de datos de cada periodo.

In [21]:
def datos_periodicos_amplitud(distr,params,num_datos,periodo):
    
    num_periodos = int(num_datos/periodo)
    datos_base0 = crear_datos(distr,params,periodo)
    datos_base = datos_base0
    for i in range(0,num_periodos-1):
        datos_base=np.concatenate((datos_base0,datos_base))
    return datos_base

Parámetros:
    - distr, params : indica la distribución que van a seguir los datos en cada periodo y los parámetros de esta distribución.
    - num_datos: indica el número de datos a crear.
    - num_periodos: indica el número de periodos

In [32]:
def datos_periodicos_cantidad(distr,params,num_datos,num_periodos):
    periodo = int(num_datos/num_periodos)
    datos_base0 = crear_datos(distr,params,periodo)
    datos_base = datos_base0
    for i in range(0,num_periodos-1):
        datos_base=np.concatenate((datos_base0,datos_base))
    return datos_base

MODELOS AUTORREGRESIVOS

Parámetros:
    - c, phi, a : parámetros del modelo autorregresivo. Nota: a guarda los valores iniciales en sus primeras posiciones.
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de datos de a ,en el caso de que pasemos como parámetro un array vacío. Indica la distribución con la que se inicializará a y los parámetros de esta distribución 

In [44]:
def modelo_AR(c,phi,distr,params,num_datos,a=[]):
    
    orden = len(phi)
    
    if len(a)==0: 
        a = crear_datos(distr,params,num_datos)
    
    datos = np.zeros(num_datos)
    
    for k in range(0,orden):
        datos[k] = a[k]
            
    for i in range(orden,num_datos):
        datos [i]= c + a[i]
        for j in range (1,orden+1):
            datos[i] = datos[i] + phi[j-1]*datos[i-j]
    
    return datos

MODELOS DE MEDIAS MOVILES

Parámetros:
    - c, teta, a : parámetros del modelo de medias móviles. Nota: a guarda los valores iniciales en sus primeras posiciones
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a, en el caso de que pasemos como parámetro un array vacío. Indica la distribución con la que se inicializará a y los parámetros de esta distribución.

In [43]:
def modelo_MA(c,teta,distr,params,num_datos,a=[]):
    
    orden = len(teta)
    
    if len(a)==0:  
        a = crear_datos(distr,params,num_datos)
    
    datos = np.zeros(num_datos)
    for i in range(0,orden):
        datos[i]= a[i]
            
    for i in range(orden,num_datos):
        datos[i] = c + a[i]
        for j in range (1,orden+1):
            datos[i]= datos[i] + teta[j-1]*a[i-j]
            
    return datos

MODELO ARMA 

Parámetros:
    - c, phi, teta, a : parámetros del modelo de autorregresivo y de medias móviles. Nota: a guarda los valores iniciales en sus primeras posiciones
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a, en el caso de que pasemos como parámetro un array vacío. Indica la distribución con la que se inicializará a y los parámetros de esta distribución.

In [49]:
def modelo_ARMA(c,phi,teta,distr,params,num_datos,a=[]):
    
    if len(a)==0:  
        a = crear_datos(distr,params,num_datos)
    
    datos = np.zeros(num_datos)
    p = len(phi)
    q = len(teta)   
    maxi = max(p,q)
    
    for k in range(0,maxi):
        datos[k] = a[k]
            
    for i in range(maxi,num_datos):
        datos[i] = c + a[i]
            
        for j in range (1,p+1):
            datos[i]= datos[i] + phi[j-1]*datos[i-j]
                
        for k in range(1,q+1):
            datos[i] = datos[i] + teta[k-1]*a[i-k]   
                
    return datos
    

MODELOS ESTACIONALES 

AUTORREGRESIVOS

Parámetros:
    - c, phi, a: parámetros del modelo autorregresivo. Nota: a guarda los valores iniciales en sus primeras posiciones
    - s: estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a, en el caso de que pasemos como parámetro un array vacío. Indica la distribución con la que se inicializará a y los parámetros de esta distribución.

In [56]:
def modelo_AR_estacional(c,phi,s,distr,params,num_datos,a=[]):
    
    if len(a)==0: 
        a = crear_datos(distr,params,num_datos)
    
    datos = np.zeros(num_datos)
    orden = len(phi)
    for k in range(0,s*orden):
        datos[k] = a[k]
            
    for i in range(orden*s,num_datos):
        datos [i]= c + a[i]
        for j in range (1,orden+1):
            datos[i] = datos[i] + phi[j-1]*datos[i-j*s]
    
    return datos

MEDIAS MÓVILES

Parámetros:
    - c, teta, a : parámetros del modelo de medias móviles. Nota: a guarda los valores iniciales en sus primeras posiciones
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a en el caso de que no lo pasemos como parámetro. Indica la distribución con la que se inicializará a y los parámetros de esta distribución.

In [59]:
def modelo_MA_estacional(c,teta,s,distr,params,num_datos,a=[]):
    
    if len(a)==0: 
        a = crear_datos(distr,params,num_datos)
    
    datos = np.zeros(num_datos)
    orden = len(teta)
    
    for i in range(0,s*orden):
        datos[i]= a[i]
            
    for i in range(s*orden,num_datos):
        datos[i] = c + a[i]
        for j in range (1,orden+1):
            datos[i]= datos[i] + teta[j-1]*a[i-j*s]
                
    return datos

ARMA

Parámetros:
    - c, phi, teta, a : parámetros del modelo de autorregresivo y de medias móviles. Nota: a guarda los valores iniciales en sus primeras posiciones
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a, en el caso de que no lo pasemos como parámetro. Indica la distribución con la que se inicializará a y los parámetros de esta distribución.

In [75]:
def modelo_ARMA_estacional(c,phi,teta,s,distr,params,num_datos,a=[]):
    
    if len(a)==0:  
        a = crear_datos(distr,params,num_datos)
    
    datos = np.zeros(num_datos)
    p = len(phi)
    q = len(teta)
    maxi = max(p,q)
    
    for k in range(0,s*maxi):
        datos[k] = a[k]
            
    for i in range(s*maxi,num_datos):
        datos[i] = c + a[i]
            
        for j in range (1,p+1):
            datos[i]= datos[i] + phi[j-1]*datos[i-j*s]
                
        for k in range(1,q+1):
            datos[i] = datos[i] + teta[k-1]*a[i-k*s]   
                
    return datos

Parámetros:
    - c, phi, teta,a : parámetros del modelo de autorregresivo y de medias móviles. Nota: a guarda los valores iniciales en sus primeras posiciones.
    - p y q : órdenes del modelo. Casos: 
        1. p=0 : modelo de medias móviles.
        2. q=0 : modelo autorregresivos.
        3. p>0 y q>0 : modelo ARMA.
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación). Casos:
        1. s>0: modelo estacional.
        2. s=0: modelo no estacional
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a en el caso de que pasemos como parámetro un array vacío. Indica la distribución con la que se inicializará a y los parámetros de esta distribución.

In [110]:
def creacion_modelos_ARMA(c,distr,params,num_datos,s=0,phi=[],teta=[],a=[]):
    
    p = len(phi)
    q = len(teta)
    
    if s == 0:
        if q == 0:
            datos = modelo_AR(c,phi,distr,params,num_datos,a)
            
        elif p == 0: 
            datos = modelo_MA(c,teta,distr,params,num_datos,a)
        else:
            datos = modelo_ARMA(c,phi,teta,distr,params,num_datos,a)
    else :
        if q == 0:
            datos = modelo_AR_estacional(c,phi,s,distr,params,num_datos,a)
            
        elif p == 0: 
            datos = modelo_MA_estacional(c,teta,s,distr,params,num_datos,a)
        else:
            datos = modelo_ARMA_estacional(c,phi,teta,s,distr,params,num_datos,a)
            
    return datos 

DRIFT CAMBIO DISTRIBUCIONES:

TIPOS DE DRIFT:
    1. CAMBIO DE DISTRIBUCIONES CREADAS CON CREAR_DATOS
    2. CAMBIO DE DISTRIBUCIÓN CREAR_DATOS A MODELOS ARMA
    3. CAMBIO DE DISTRIBUCIÓN CREAR_DATOS A PERIÓDICOS
    4. CAMBIO DE DISTRIBUCIÓN CREAR_DATOS A TENDENCIAS DETERMINISTAS 
    5. CAMBIO DE MODELO ARMA A MODELO ARMA
    6. CAMBIO DE MODELO ARMA A DISTRIBUCIÓN CREAR DATOS 
    7. CAMBIO DE MODELO ARMA A PERIODICOS 
    8. CAMBIO DE MODELO ARMA A TENDENCIAS DETERMINISTAS
    9. CAMBIO DE PERIÓDICOS A PERIÓDICOS
    10. CAMBIO DE PERIÓDICOS A DISTRIBUCIONES DE CREAR_DATOS
    11. CAMBIO DE PERIODICOS A MODELOS ARMA
    12. CAMBIO DE PERIÓDICOS A TENDENCIAS DETERMINISTAS
    13. CAMBIO DE TENDENCIAS DETERMINISTAS A TENDENCIAS DETERMINISTAS 
    14. CAMBIO DE TENDENCIAS DETERMINISTAS A DISTRIBUCIONES CON CREAR_DATOS
    15. CAMBIO DE TENDENCIAS DETERMINISTAS A MODELOS ARMA
    16. CAMBIO DE TENDENCIAS DETERMINISTAS A PERIÓDICOS

Parámetros:
    - params1,params2 : parámetros necesarios para crear los datos pervios y posteriores al drift. Casos:
        1. DISTRIBUCIONES CREADAS CON CREAR DATOS: los parámetros son distr y parametros que indica la distribución y los parámetros de la distribución.
        2. MODELOS ARMA: los parámetros son los del modelo arma,es decir, c, phi, teta, a, s, distr y params.
        3. DATOS PERIODICOS: los parametros son el tipo (1 indica que se ha pasado la amplitud del periodo y 2 la cantidad de periodos) y distr, params y p (indica el número de periodos o su amplitud) necesarios para crear los datos periódicos.
        4. TENDENCIA DETERMINISTA: los parámetros son el tipo de tendencia, los parámetros del modelo y el coeficiente de error.  
    - tipo: tipo de drift.
    - num_drift: número de datos previos al drift.
    - num_datos: número de datos totales.

In [99]:
params1= [1,[4,2]]

In [97]:
params2=[5,1,[3,1],3,[0.3,0.4],[0.1],[]]

In [130]:
par = [[5,0.6,0.1],2,0.1]

In [104]:
params3=[2,1,[7,1.5],5]

In [106]:
params4=[[5,0.6],1,0.1]

In [134]:
crear_drift(params4,params2,15,60,150)

IndexError: list index out of range

In [128]:
def crear_drift(params1,params2,tipo,num_drift,num_datos):
    
    if tipo==1:
        distr1=params1[0]
        parametros1=params1[1]
        datos1 = crear_datos(distr1,parametros1,num_drift)
        
        distr2=params2[0]
        parametros2=params2[1]
        datos2 = crear_datos(distr2,parametros2,num_datos-num_drift)
        
    elif tipo==2:
        distr1=params1[0]
        parametros1=params1[1]
        datos1 = crear_datos(distr1,parametros1,num_drift)
        
        c = params2[0]
        distr = params2[1]
        parametros2 = params2[2]
        s = params2[3]
        phi = params2[4]
        teta = params2[5]
        a = params2[6]
        datos2 = creacion_modelos_ARMA(c,distr,parametros2,num_datos-num_drift,s,phi,teta,a)
    
    elif tipo==3:
        distr1=params1[0]
        parametros1=params1[1]
        datos1 = crear_datos(distr1,parametros1,num_drift)
        
        tipo2 = params2[0]
        distr,parametros2,p = params2[1],params2[2],params2[3]
        if tipo2==1:
            datos2=datos_periodicos_amplitud(distr,parametros2,num_datos-num_drift,p)
        elif tipo2==2:
            datos2=datos_periodicos_cantidad(distr,parametros2,num_datos-num_drift,p)
            
    elif tipo==4:
        distr1=params1[0]
        parametros1=params1[1]
        datos1 = crear_datos(distr1,parametros1,num_drift)
        
        parametros2,tipo2,coef_error = params2[0],params2[1],params2[2]
        datos2=tendencia_det(parametros2,tipo2,num_datos-num_drift,coef_error)
    
    elif tipo==5:
        c = params1[0]
        distr = params1[1]
        parametros1 = params1[2]
        s = params1[3]
        phi = params1[4]
        teta = params1[5]
        a = params1[6]
        datos1 = creacion_modelos_ARMA(c,distr,parametros1,num_drift,s,phi,teta,a)
        
        c2 = params2[0]
        distr2 = params2[1]
        parametros2 = params2[2]
        s2 = params2[3]
        phi2 = params2[4]
        teta2 = params2[5]
        a2 = params2[6]
        datos2 = creacion_modelos_ARMA(c2,distr2,parametros2,num_datos-num_drift,s2,phi2,teta2,a2)
        
    elif tipo==6: 
        
        c = params1[0]
        distr = params1[1]
        parametros1 = params1[2]
        s = params1[3]
        phi = params1[4]
        teta = params1[5]
        a = params1[6]
        datos1 = creacion_modelos_ARMA(c,distr,parametros1,num_drift,s,phi,teta,a)
        
        distr2=params2[0]
        parametros2=params2[1]
        datos2 = crear_datos(distr2,parametros2,num_datos - num_drift)
  
        
    elif tipo==7: 
        c = params1[0]
        distr = params1[1]
        parametros1 = params1[2]
        s = params1[3]
        phi = params1[4]
        teta = params1[5]
        a = params1[6]
        datos1 = creacion_modelos_ARMA(c,distr,parametros1,num_drift,s,phi,teta,a)

        tipo2 = params2[0]
        distr,param,p = params2[1],params2[2],params2[3]
        if tipo2==1:
            datos2=datos_periodicos_amplitud(distr,param,num_datos-num_drift,p)
        elif tipo2==2:
            datos2=datos_periodicos_cantidad(distr,param,num_datos-num_drift,p)
            
    elif tipo==8:
        
        c = params1[0]
        distr = params1[1]
        parametros1 = params1[2]
        s = params1[3]
        phi = params1[4]
        teta = params1[5]
        a = params1[6]
        datos1 = creacion_modelos_ARMA(c,distr,parametros1,num_drift,s,phi,teta,a)
    
        param,tipo2,coef_error = params2[0],params2[1],params2[2]
        datos2 = tendencia_det(param,tipo2,num_datos-num_drift,coef_error)  
         
    elif tipo==9: 
        tipo1 = params1[0]
        distr,params,p = params1[1],params1[2],params1[3]
        if tipo1==1:
            datos1=datos_periodicos_amplitud(distr,params,num_drift,p)
        elif tipo1==2:
            datos1=datos_periodicos_cantidad(distr,params,num_drift,p)
            
        tipo2 = params2[0]
        distr,param,p = params2[1],params2[2],params2[3]
        if tipo2==1:
            datos2=datos_periodicos_amplitud(distr,param,num_datos-num_drift,p)
        elif tipo2==2:
            datos2=datos_periodicos_cantidad(distr,param,num_datos-num_drift,p)
    
    elif tipo==10:
        tipo1 = params1[0]
        distr,params,p = params1[1],params1[2],params1[3]
        if tipo1==1:
            datos1=datos_periodicos_amplitud(distr,params,num_drift,p)
        elif tipo1==2:
            datos1=datos_periodicos_cantidad(distr,params,num_drift,p)
            
        distr2=params2[0]
        parametros2=params2[1]
        datos2 = crear_datos(distr2,parametros2,num_datos - num_drift)   
        
    elif tipo == 11:
        tipo1 = params1[0]
        distr,params,p = params1[1],params1[2],params1[3]
        if tipo1==1:
            datos1=datos_periodicos_amplitud(distr,params,num_drift,p)
        elif tipo1==2:
            datos1=datos_periodicos_cantidad(distr,params,num_drift,p)
            
        c2 = params2[0]
        distr2 = params2[1]
        parametros2 = params2[2]
        s2 = params2[3]
        phi2 = params2[4]
        teta2 = params2[5]
        a2 = params2[6]
        datos2 = creacion_modelos_ARMA(c2,distr2,parametros2,num_datos-num_drift,s2,phi2,teta2,a2)  
          
    elif tipo==12:
        tipo1 = params1[0]
        distr,params,p = params1[1],params1[2],params1[3]
        if tipo1==1:
            datos1=datos_periodicos_amplitud(distr,params,num_drift,p)
        elif tipo1==2:
            datos1=datos_periodicos_cantidad(distr,params,num_drift,p)
            
        param,tipo2,coef_error = params2[0],params2[1],params2[2]
        datos2=tendencia_det(param,tipo2,num_datos-num_drift,coef_error)  
           
    elif tipo==13:
        params,tipo1,coef_error = params1[0],params1[1],params1[2]
        datos1=tendencia_det(params,tipo1,num_drift,coef_error) 
        
        param,tipo2,coef_error2 = params2[0],params2[1],params2[2]
        datos2=tendencia_det(param,tipo2,num_datos-num_drift,coef_error2) 
         
    elif tipo==14:
        params,tipo1,coef_error = params1[0],params1[1],params1[2]
        datos1=tendencia_det(params,tipo1,num_drift,coef_error) 
        
        distr2=params2[0]
        parametros2=params2[1]
        datos2 = crear_datos(distr2,parametros2,num_datos - num_drift)
        
    elif tipo==15:
        params,tipo1,coef_error = params1[0],params1[1],params1[2]
        datos1=tendencia_det(params,tipo1,num_drift,coef_error) 
        
        c2 = params2[0]
        distr2 = params2[1]
        parametros2 = params2[2]
        s2 = params2[3]
        phi2 = params2[4]
        teta2 = params2[5]
        a2 = params2[6]
        datos2 = creacion_modelos_ARMA(c2,distr2,parametros2,num_datos-num_drift,s2,phi2,teta2,a2)
        
    elif tipo==16: 
        params,tipo1,coef_error = params1[0],params1[1],params1[2]
        datos1=tendencia_det(params,tipo1,num_drift,coef_error) 
        
        tipo2 = params2[0]
        distr,param,p = params2[1],params2[2],params2[3]
        if tipo2==1:
            datos2=datos_periodicos_amplitud(distr,param,num_datos-num_drift,p)
        elif tipo2==2:
            datos2=datos_periodicos_cantidad(distr,param,num_datos-num_drift,p)
             
    datos = np.concatenate((datos1,datos2))
    return datos 

CREACIÓN DE DATAFRAMES

SEGÚN UN MODELO DE TENDENCIA DETERMINISTA

Parámetros: 
    - params: indica los parámetros de la tendencia.
    - tipo: Valores posibles:
        1. Tendencia determinista lineal
        2. Tendencia determinista cuadrática
        3. Tendencia determinista exponencial
    - inicio, fin: fecha inicio y fin de la serie temporal.
    - columna: nombre de la columna
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds
    - coef_error: indica el coeficiente de error (a mayor coeficiente, menor estacionariedad). Si el coef_error=0, los datos son estacionarios. 

In [25]:
def crear_df_fin_tend_det(inicio,fin,freq,columna,params,tipo,coef_error=0):
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    datos = tendencia_det(params,tipo,num_datos,coef_error)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df

Parámetros: 
    - params: indica los parámetros de la tendencia.
    - tipo: Valores posibles:
        1. Tendencia determinista lineal
        2. Tendencia determinista cuadrática
        3. Tendencia determinista exponencial
    - inicio, periodos: fecha inicio y nº de periodos.
    - columna: nombre de la columna 
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds
    - coef_error: indica el coeficiente de error (a mayor coeficiente, menor estacionariedad). Si el coef_error=0, los datos son estacionarios. 

In [26]:
def crear_df_periodos_tend_det(inicio,periodos,freq,columna,params,tipo,coef_error=0):
    indice = series_periodos(inicio,periodos,freq)
    num_datos = indice.size
    datos = tendencia_det(params,tipo,num_datos,coef_error)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df

SEGÚN EL MODELO DE DISTRIBUCIONES DE CREAR_DATOS:

Parámetros:
- inicio, fin: fecha inicio y fin de la serie temporal.
- columna: nombre de la columna
- freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds
- Distr, params : Indica la distribución a seguir y los parámetros de esta distribución Casos:
    1. Los parámetros son la media y la desv_típica de los datos. Sigue una distribucón normal.
    2. Los parámetros son p(probabilidad éxito) y n(nº pruebas) de una distribución binomial. Además podemos añadir otro parámetro que indica la localización de los datos.
    3. El parámetro es mu de una distribución poisson. Además podemos añadir otro parámetro que indica la localización de los datos.
    4. El parámetro es p(probabilidad éxito) de una distribución geométrica. Además podemos añadir otro parámetro que indica la localización de los datos.
    5. Los parámetros son M(tamaño población), n (nº aciertos) y N (tamño muestra)de una distribución hipergeométrica. Además podemos añadir otro parámetro que indica la localización de los datos.
    6. El parámetro es el valor constante que toma los datos.
    7. Sigue una distribución uniforme, puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    8. El parámetro es s de una distribución lognormal. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    9. Sigue una distribución exponencial, puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    10. El parámetro es a de una distribución gamma. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    11. Los parámetros son a y b de una distribución beta. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    12. El parámetro es a de una distribución gamma. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    13. El parámetro es df de una distribución chi cuadrado. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    14. El parámetro es t de una distribución t-student. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    15. El parámetro es b de una distribución pareto. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    16. Sigue una distribución lineal descendente cuyos parámetros son el dato inicial y la pendiente (en valor absoluto), si llegamos a números negativos nos mantenemos como cte 0.
    17. Sigue una distribución lineal ascendente cuyos parámetros son el dato inicial y la pendiente. 
    18. Obtienes números aleatorios cuyos valores están entre el primer parámetro y el segundo.

In [27]:
def crear_df_fin_datos(inicio,fin,freq,columna,distr,params):
    
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    datos = crear_datos(distr,params,num_datos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df 

Parámetros:
- inicio, periodos: fecha inicio y nº periodos de la serie temporal. 
- columna: nombre de la columna
- freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds
- Distr, params : Indica la distribución a seguir y los parámetros de esta distribución Casos:
    1. Los parámetros son la media y la desv_típica de los datos. Sigue una distribucón normal.
    2. Los parámetros son p(probabilidad éxito) y n(nº pruebas) de una distribución binomial. Además podemos añadir otro parámetro que indica la localización de los datos.
    3. El parámetro es mu de una distribución poisson. Además podemos añadir otro parámetro que indica la localización de los datos.
    4. El parámetro es p(probabilidad éxito) de una distribución geométrica. Además podemos añadir otro parámetro que indica la localización de los datos.
    5. Los parámetros son M(tamaño población), n (nº aciertos) y N (tamño muestra)de una distribución hipergeométrica. Además podemos añadir otro parámetro que indica la localización de los datos.
    6. El parámetro es el valor constante que toma los datos.
    7. Sigue una distribución uniforme, puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    8. El parámetro es s de una distribución lognormal. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    9. Sigue una distribución exponencial, puede no tener parámetros, indicar solo la localización o indicar la localización y escala. 
    10. El parámetro es a de una distribución gamma. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    11. Los parámetros son a y b de una distribución beta. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    12. El parámetro es a de una distribución gamma. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    13. El parámetro es df de una distribución chi cuadrado. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    14. El parámetro es t de una distribución t-student. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    15. El parámetro es b de una distribución pareto. Además podemos añadir otros parámetros que indican la localización o localización y escala de los datos.
    16. Sigue una distribución lineal descendente cuyos parámetros son el dato inicial y la pendiente (en valor absoluto), si llegamos a números negativos nos mantenemos como cte 0.
    17. Sigue una distribución lineal ascendente cuyos parámetros son el dato inicial y la pendiente. 
    18. Obtienes números aleatorios cuyos valores están entre el primer parámetro y el segundo.

In [28]:
def crear_df_periodos_datos(inicio,periodos,freq,columna,distr,params):
    
    indice = series_periodos(inicio,periodos,freq)
    datos = crear_datos(distr,params,periodos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df 

SEGÚN MODELOS PERIÓDICOS:

Parámetros: 
    - inicio, fin: fecha de inicio y fin de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna 
    - distr, params: indica la distribución y los parámetros que siguen los datos
    - p: indica la cantidad de periodos/ la amplitud de los periodos
    - tipo: valores posibles
        1. p = amplitud de periodo
        2. p = número de periodos

In [29]:
def crear_df_fin_periodicos(inicio,fin,freq,columna,distr,params,p,tipo):
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    if tipo==1:
        datos = datos_periodicos_amplitud(distr,params,num_datos,p)
    elif tipo ==2:
        datos=datos_periodicos_cantidad(distr,params,num_datos,p)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df 

Parámetros: 
    - inicio, periodos: fecha de inicio y nº de periodos de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna 
    - distr, params: indica la distribución y los parámetros que siguen los datos
    - p: indica la cantidad de periodos/ la amplitud de los periodos
    - tipo: valores posibles
        1. p = amplitud de periodo
        2. p = número de periodos

In [30]:
def crear_df_periodo_periodicos(inicio,periodos,freq,columna,distr,params,p,tipo):
    indice = series_periodos(inicio,periodos,freq)
    num_datos = indice.size
    if tipo==1:
        datos = datos_periodicos_amplitud(distr,params,num_datos,p)
    elif tipo==2:
        datos=datos_periodicos_cantidad(distr,params,num_datos,p)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df 

SEGÚN MODELOS AUTORREGRESIVOS Y DE MEDIAS MÓVILES: 

Parámetros: 
    - c, phi, teta,a : parámetros del modelo de autorregresivo y de medias móviles. Nota: a guarda los valores iniciales en sus primeras posiciones
    - p y q : órdenes del modelo.
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a en el caso de que pasemos como parámetro un array vacío
    - inicio, fin: indica la fecha de inicio y fin de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna 

In [31]:
def crear_df_fin_ARMA(inicio,fin,freq,columna,c,phi,teta,a,p,q,s,distr,params):
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    datos=creacion_modelos_ARMA(c,phi,teta,a,p,q,s,distr,params,num_datos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df 

Parámetros:
    - c, phi, teta, a : parámetros del modelo de autorregresivo y de medias móviles. Nota: a guarda los valores iniciales en sus primeras posiciones
    - p y q : órdenes del modelo.
    - s : estacionalidad, tamaño de una estación (número de datos que hay en cada estación)
    - num_datos: numéro de datos a crear 
    - distr,params: necesarios para la creación de a en el caso de que pasemos como parámetro un array vacío
    - inicio, periodos: indica la fecha de inicio y nº de periodos de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna

In [32]:
def crear_df_periodos_ARMA(inicio,periodos,freq,columna,c,phi,teta,a,p,q,s,distr,params):
    indice = series_periodos(inicio,periodos,freq)
    datos=creacion_modelos_ARMA(c,phi,teta,a,p,q,s,distr,params,periodos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df 

MODELOS CON DRIFT 

Parámetros:
    - params1,params2 : parámetros necesarios para crear los datos pervios y posteriores al drift. Casos:
        1. DISTRIBUCIONES CREADAS CON CREAR DATOS: los parámetros son distr y parametros que indica la distribución y los parámetros de la distribución.
        2. MODELOS ARMA: los parámetros son los del modelo arma,es decir, c, phi, teta, a, p, q, s, distr y params.
        3. DATOS PERIODICOS: los parametros son el tipo (1 indica que se ha pasado la amplitud del periodo y 2 la cantidad de periodos) y distr, params y p (indica el número de periodos o su amplitud) necesarios para el crear los datos periódicos.
        4. TENDENCIA DETERMINISTA: los parámetros son el tipo de tendencia, los parámetros del modelo y el coeficiente de error.  
    - tipo: tipo de drift.
    - num_drift: número de datos previos al drift.
    - inicio,fin: indica la fecha de inicio y fin de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna

In [33]:
def crear_df_fin_DRIFT(inicio,fin,freq,columna,params1,params2,tipo,num_drift):
    indice = series_fin(inicio,fin,freq)
    num_datos = indice.size
    datos = crear_drift(params1,params2,tipo,num_drift,num_datos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df 

Parámetros:
    - params1,params2 : parámetros necesarios para crear los datos pervios y posteriores al drift. Casos:
        1. DISTRIBUCIONES CREADAS CON CREAR DATOS: los parámetros son distr y parametros que indica la distribución y los parámetros de la distribución.
        2. MODELOS ARMA: los parámetros son los del modelo arma,es decir, c, phi, teta, a, p, q, s, distr y params.
        3. DATOS PERIODICOS: los parametros son el tipo (1 indica que se ha pasado la amplitud del periodo y 2 la cantidad de periodos) y distr, params y p (indica el número de periodos o su amplitud) necesarios para el crear los datos periódicos.
        4. TENDENCIA DETERMINISTA: los parámetros son el tipo de tendencia, los parámetros del modelo y el coeficiente de error.  
    - tipo: tipo de drift.
    - num_drift: número de datos previos al drift.
    - inicio, periodos: indica la fecha de inicio y nº de periodos de la serie temporal.
    - freq: B business day frequency, D calendar day frequency, W weekly frequency, M monthly frequency, Q quarterly frequency, Y yearly frequency, h hourly frequency, min minutely frequency, s secondly frequency, ms milliseconds, us microseconds, ns nanoseconds.
    - columna: nombre de la columna

In [34]:
def crear_df_periodos_DRIFT(inicio,periodos,freq,columna,params1,params2,tipo,num_drift):
    indice = series_fin(inicio,periodos,freq)
    datos = crear_drift(params1,params2,tipo,num_drift,periodos)
    df = pd.DataFrame(data=datos,index=indice,columns=columna)
    df.plot(title='Serie Temporal')
    return df 